AUTORZY: Szymon Sroka 141312, Weronika Radzi 141303

Zadanie wykonaliśmy w wariancie ze śledzeniem kul, wykrywaniem zderzenia się dwóch dowolnych kul oraz wpadnięcia kuli do łuzy. W stworzonym algorytmie możemy wyodrębnić następujące etapy:

1. *find_objects()* - Detekcja położenie kul w pierwszej klatce filmu (czyli innymi słowy wykrycie punktów, które powinny być śledzone). Postanowliśmy wykrywać je z pomocą kilku obrazów-szablonów przedstawiających kule, oznaczonych w plikach wejściowych jako ball_1.png, ball_2.png itd. Wspomniane szablony zostały razem z pierwszą klatką filmu podane do metody cv2.matchTemplate(), która zwróciła zbiór żądanych punktów. 
2. *remove_duplicates()* - Usunięcie duplikatów punktów ze zbioru uzyskanego w punkcie 1. Za duplikaty uznajemy punkty, których wzajemna odległość euklidesowa jest mniejsza lub równa 15.
3. *detect_actions()* - Śledzenie kul i interpratacja otrzymywanych wyników. Do śledzenia ruchu optycznego użyliśmy cv2.calcOpticalFlowPyrLK(). W celu detekcji **zderzenia bil** dla każdej klatki jest obliczana macierz odległości między kulami dla aktualnej i poprzedniej klatki i w sytuacji, gdy odległość między danymi kulami w poprzedniej klatce jest większa niż wyznaczony empirycznie próg (15) i jednocześnie ta odległość w aktualnej klatce jest mniejsza niż 15, to uznajemy taką sytuację za zderzenie kul. Jako **wpadnięcie bili do łuzy** interpretujemy zniknięcie śledzonego punktu-bili z pola widzenia (a zatem zmienienie dla danej kuli wartości z 1 na 0 w macierzy st, zwracanej przez calcOpticalFlowPyrLK())

Śledzenie kul zostało zaprezentowane za pomocą otoczenia ich okręgami, podania obok tych okręgów identyfiaktorów przydzielonym danym kulom i rysowania przebytych przez nie tras. 

W przypadku wykrycia zderzenia lub wpadnięcia bili do łuzy na filmie wyświetlany jest odpowiedni napis, a film zostaje na kilkanaście klatek zatrzymany, w celu możliwości zaobserwowania danego zdarzenia. Takie zachowanie można wyłączyć, ustawiając wartość parametru ,,freeze_mode" w funkcji detect_actions() na false. 

Uzyskane efekty w ogólności uznajemy za dobre; zazwyczaj prawidłowo działa wykrywanie obiektów-bil oraz ich śledzenie, gorszą jakość uzyskiwanych wyników obserwujemy dla detekcji wpadnięcia bili do łuzy czy śledzenia ich, gdy ich prędkość jest relatywnie duża. Uważamy, że algorytm można ulepszyć poprzez tuning parametrów sterujacych przetwarzaniem obrazu, a także poprzez wprowadzenie ulepszenia polegającego na detekcji położenia łuz i wykorzystaniu uzyskanych informacji do wykrywania zniknięcia bil (na podstawie wzajemnego położenia bil i wyżej wspomnianych łuz), zamiast polegania wyłącznie na wynikach zwracanych przez calcOpticalFlowPyrLK(). Przewidujemy również, że wybór filmów z większą liczbą FPS niż zastosowane 30 klatek na sekundę wpłynąłby korzystnie na efekty przetwarzania.

In [ ]:
# necessary imports
import cv2 
import numpy as np
from ipywidgets import Video
from google.colab.patches import cv2_imshow

In [ ]:
# finding points to follow using templates
# returns p0 and old_frame
def find_objects(vid, filenames):
  ret, old_frame = vid.read()
  old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

  loc_x, loc_y = np.array([]), np.array([])
  for filename in filenames: # iterating over templates
      template = cv2.imread(filename,0)
      w, h = template.shape[::-1]
      
      res = cv2.matchTemplate(old_gray,template,cv2.TM_CCOEFF_NORMED)
      threshold = 0.8

      loc_x = np.concatenate((loc_x, np.where( res >= threshold)[1] ))
      loc_y = np.concatenate((loc_y, np.where( res >= threshold)[0] ))
      
  return np.stack((loc_x.flatten()+h//2,loc_y.flatten()+w//2),axis=1).astype(np.float32), old_frame 


# determines if two points a and b differ (with tolerance)
def differ(a,b, tolerance = 15):
  return False if np.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2) <= tolerance else True


# removes duplicates from set of points (with tolerance)
def remove_duplicates(points):
  result = []
  for point in points:
      if all(differ(point,other) for other in result):
          result.append(point)
  return result


In [ ]:
from scipy.spatial.distance import squareform, pdist

def which_collided(p0, p1):

    old_dist, new_dist = squareform(pdist(p0.reshape(p0.shape[0],2))), squareform(pdist(p1.reshape(p1.shape[0],2)))

    res = []
    for i,j in np.ndindex(old_dist.shape):
      if old_dist [i,j] > 15 > new_dist[i,j]:
        if [j,i] not in res:
          res.append([i,j])
    return res 

def detect_actions(vid, p0, lk_params, old_frame, filename, freeze_mode=True):
 
  color = np.random.randint(0,255,(1000,3))  # some random colors for marking billard balls tracks

  result_filename = filename.split(".")[0]+"_result.mp4"
  
  result_video = cv2.VideoWriter(result_filename, 
                                 cv2.VideoWriter_fourcc(*'DIVX'), 
                                 int(vid.get(cv2.CAP_PROP_FPS)), 
                                 (int(vid.get(3)), int(vid.get(4))))

  mask = np.zeros_like(old_frame)

  old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

  vid.set(cv2.CAP_PROP_POS_FRAMES, 0)

  st_old =  None

  while vid.isOpened():
    
    ret, frame = vid.read()

    if ret:
      frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

      if p0 is None or p1 is None: break

      collisions, disappears = which_collided(p0, p1), []
      
      if st_old is not None:
        for i in range(len(p0)):
            if st_old[i] == 1 and st[i] == 0: 
              disappears.append(i)
      
      if len(disappears) > 0:   ball_text = cv2.putText(frame, 
                                f"Ball no. {disappears} disappeared",
                                (30, int(0.95*int(vid.get(4)))),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1, (255,255,255), 2)

      if p1 is not None:
        
        good_new = p1[st==1]
        good_old = p0[st==1]

      for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        
        mask = cv2.line(mask, (int(a),int(b)),(int(c),int(d)), (color[i].tolist()), 2)
        ball_text = cv2.putText(frame, 
                                f"{i}{'(collision)' if i in [a for a, _ in collisions] else ''}",
                                (int(c)+25,int(d)),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.5, (255,255,255), 1)
        frame = cv2.circle(frame,(int(a),int(b)),20,(color[i].tolist()), 2)
      
      old_gray = frame_gray.copy()
      p0 = good_new.reshape(-1,1,2)
      st_old = st

      result_video.write(cv2.add(frame, mask, ball_text))

      if (len(collisions) > 0 or len(disappears) > 0) and freeze_mode:
        for _ in range(35): result_video.write(frame)
    
    else:
        break

  result_video.release()
  return result_filename

In [ ]:
def get_result(filename, lk_params, template_filenames):
  vid = cv2.VideoCapture(filename)
  p0, old_frame = find_objects(vid,templates_filenames)
  p0 = remove_duplicates(p0)
  p0 = np.array([np.array([x]) for x in p0])  # prepare found points before passing them to detect_actions()
  return detect_actions(vid, p0, lk_params, old_frame, filename)


In [ ]:
# defining params
templates_filenames = ['ball.png','ball_1.png','ball_2.png', 'ball_3.png']

lk_params = dict(winSize=(10,10),
                 maxLevel=1,
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100,  1))

In [ ]:
filename = "(3).mp4"
result_filename = get_result(filename, lk_params, templates_filenames)
!ffmpeg -hide_banner -loglevel error -i '{result_filename}' -y xyz.mp4
Video.from_file(f"./xyz.mp4")

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x03\x95\xe6mdat\x0…

In [ ]:
filename = "(2).mp4"
result_filename = get_result(filename, lk_params, templates_filenames)
!ffmpeg -hide_banner -loglevel error -i '{result_filename}' -y xyz.mp4
Video.from_file(f"./xyz.mp4")

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x05\xd1\tmdat\x00\…

In [ ]:
filename = "(1).mp4"
result_filename = get_result(filename, lk_params, templates_filenames)
!ffmpeg -hide_banner -loglevel error -i '{result_filename}' -y xyz.mp4
Video.from_file(f"./xyz.mp4")

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x07\xc3\xc2mdat\x0…